In [1]:
import pandas
from IPython.display import clear_output

In [2]:
from classes import Question
import look_up_utils
import createq_utils
from display_utils import display_question

## Set parameter values

In [3]:
frames = ['mass_shootings',
          'mass_shootings_2013',
          'mass_shootings_2014',
          'mass_shootings_2015']
confusion_tuple = ('location', 'time')
min_confusion = 3

## Load frames

In [4]:
df = pandas.concat([pandas.read_pickle('../EventRegistries/GunViolence/frames/' + frame)
                    for frame in frames])

## Load look_up

In [5]:
look_up, parameters2incident_uris = look_up_utils.create_look_up(frames)

## Load questions

In [6]:
candidates=createq_utils.lookup_and_merge(look_up, 
                                          parameters2incident_uris,
                                          confusion_tuple,
                                          min_confusion,
                                          df,
                                          debug=True,
                                          inspect_one=False) 
print({c.granularity for c in candidates})

{('city', 'year')}


## Display a question

In [7]:
print('the available attributes are:\n')
for candidate in candidates:
    for prop in dir(candidate):
        if all([not prop.startswith('_'),
                prop not in {'debug', 'noise_df', 'answer_df'}]):
            print(prop)
    break

the available attributes are:

a_avg_date_spread
a_avg_num_sources
a_sources
answer
answer_incident_uris
confusion_factors
granularity
location_confusion
meanings
n2s_ratio
n_avg_date_spread
n_avg_num_sources
n_sources
noise_incident_uris
oa_info
participant_confusion
q_id
question
sf
time_confusion


In [8]:
wanted_attrs = [
    'q_id',
    'question',
    'answer_incident_uris',
    'meanings',
    'answer',
    'n2s_ratio',
    'location_confusion',
    'time_confusion',
    'participant_confusion',
    'a_avg_num_sources',
    'a_avg_date_spread',
    'n_avg_num_sources',
    'n_avg_date_spread',
]

for candidate in candidates:
    clear_output()
    display_question(candidate, wanted_attrs)
    input('next?')

,0
q_id,2
question,How many events happened in 2016 in Jackson ?
answer_incident_uris,"{'563630', '688429', '601119', '686312'}"
meanings,"defaultdict(, {(('Mississippi', 'Jackson'), ('2016',)): {'688429'}, (('Georgia', 'Jackson'), ('2016',)): {'563630', '686312'}, (('Michigan', 'Jackson'), ('2016',)): {'601119'}})"
answer,4
n2s_ratio,96.25
location_confusion,True
time_confusion,True
participant_confusion,False
a_avg_num_sources,3.5


next?
